In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Scrapping Data

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print( tabulate(df[0], headers='keys', tablefmt='psql') )

+-----+---------------+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|     | Postal code   | Borough          | Neighborhood                                                                                                                                  |
|-----+---------------+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | M1A           | Not assigned     | nan                                                                                                                                           |
|   1 | M2A           | Not assigned     | nan                                                                                                                                           |
|   2 | M3A           | North York       | Parkwoods             

## Making Dataframe from raw Data

In [43]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

In [44]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [45]:
df.dtypes

Postal code     object
Borough         object
Neighborhood    object
dtype: object

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


## Splitting Data so that it is easily accessable

In [47]:
df["Neighborhood"]= df["Neighborhood"].str.split("/") 
  
# df display 
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal code     180 non-null object
Borough         180 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


## droping not assigned Values

In [49]:
empty = 'Not assigned'
df = df[(df['Postal code'] != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [42]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


### rename column names, as it was disturbing

In [65]:
df.columns=['Postalcode','Borough','Neighborhood']

In [66]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


In [50]:
df.shape

(103, 3)

# Installing GeoCoder

In [52]:
!pip install geocoder

     |████████████████████████████████| 98 kB 2.0 MB/s eta 0:00:011


## But Unfortunately, It was not able to fetch records. so read from csv

In [59]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Again rename column Names

In [63]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [64]:
df_lon_lat.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging Data so that final Dataframe Comes in

In [70]:
df_final = pd.merge(df,df_lon_lat[['Postalcode','Latitude', 'Longitude']],on='Postalcode')
df_final.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park , Harbourfront]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor , Lawrence Heights]",43.718518,-79.464763
4,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]",43.662301,-79.389494


# 3rd Part

## using foursqaure to get venues

In [73]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [74]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [76]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [77]:
CLIENT_ID = 'MIF2SPKPEZP0YIUKZMPBLCA3R4ESWCFJDYPFTPHV4PTFVXIO' # your Foursquare ID
CLIENT_SECRET = '5UOPEX5O43CRW0TZWYYOXST2VJBTPKIXAJWJ2TJVH3S23ZEK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MIF2SPKPEZP0YIUKZMPBLCA3R4ESWCFJDYPFTPHV4PTFVXIO
CLIENT_SECRET:5UOPEX5O43CRW0TZWYYOXST2VJBTPKIXAJWJ2TJVH3S23ZEK


In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                  )

['Parkwoods']
['Victoria Village']
['Regent Park ', ' Harbourfront']
['Lawrence Manor ', ' Lawrence Heights']
["Queen's Park ", ' Ontario Provincial Government']
['Islington Avenue']
['Malvern ', ' Rouge']
['Don Mills']
['Parkview Hill ', ' Woodbine Gardens']
['Garden District, Ryerson']
['Glencairn']
['West Deane Park ', ' Princess Gardens ', ' Martin Grove ', ' Islington ', ' Cloverdale']
['Rouge Hill ', ' Port Union ', ' Highland Creek']
['Don Mills']
['Woodbine Heights']
['St. James Town']
['Humewood-Cedarvale']
['Eringate ', ' Bloordale Gardens ', ' Old Burnhamthorpe ', ' Markland Wood']
['Guildwood ', ' Morningside ', ' West Hill']
['The Beaches']
['Berczy Park']
['Caledonia-Fairbanks']
['Woburn']
['Leaside']
['Central Bay Street']
['Christie']
['Cedarbrae']
['Hillcrest Village']
['Bathurst Manor ', ' Wilson Heights ', ' Downsview North']
['Thorncliffe Park']
['Richmond ', ' Adelaide ', ' King']
['Dufferin ', ' Dovercourt Village']
['Scarborough Village']
['Fairview ', ' Henry Fa

In [82]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,[Parkwoods],43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,[Parkwoods],43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,[Victoria Village],43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,[Victoria Village],43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,[Victoria Village],43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,[Victoria Village],43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,[Victoria Village],43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"[Regent Park , Harbourfront]",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"[Regent Park , Harbourfront]",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"[Regent Park , Harbourfront]",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [83]:
toronto_venues.shape

(1345, 7)